Install required libraries

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git
!pip uninstall -y torch torchvision torchaudio
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install bitsandbytes accelerate
!pip install -q fastapi uvicorn nest-asyncio pyngrok transformers


Quantize the Airavata LLM and save it

In [ ]:
import torch
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,        # or load_in_8bit=True for 8-bit quant
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16  # or float16 depending on your GPU
)

# Load tokenizer and quantized model
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("ai4bharat/Airavata")
model = AutoModelForCausalLM.from_pretrained(
    "ai4bharat/Airavata",
    quantization_config=bnb_config,
    device_map='auto'
)

# Save both the tokenizer and the quantized model
save_dir = "./Quantized_Airavata"

# Save model
model.save_pretrained(save_dir)

# Save tokenizer
tokenizer.save_pretrained(save_dir)


create app.py file

In [ ]:
%%writefile app.py
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch

app = FastAPI()

model_path = "/content/Quantized_Airavata"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype=torch.bfloat16
)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Root endpoint (fixes "Not Found" at /)
@app.get("/")
def root():
    return {"message": "✅ Airavata FastAPI server is running. Use /docs to try it."}

# Input model
class GenerateRequest(BaseModel):
    prompt: str
    max_new_tokens: int = 100

# POST endpoint for text generation
@app.post("/generate")
async def generate(req: GenerateRequest):
    output = pipe(
        req.prompt,
        max_new_tokens=req.max_new_tokens,
        do_sample=True,
        temperature=0.7,
        top_p=0.9
    )
    return {"generated_text": output[0]["generated_text"]}


In the below cell replace your_authtoken_here with your ngrok authtoken

In [ ]:
!ngrok config add-authtoken your_authtoken_here


In [ ]:
import nest_asyncio
import uvicorn
from pyngrok import ngrok
import threading

nest_asyncio.apply()

# Start tunnel
public_url = ngrok.connect(8000)
print("🔗 Public URL:", public_url)

# Start server in a thread
def run():
    uvicorn.run("app:app", host="0.0.0.0", port=8000)

thread = threading.Thread(target=run)
thread.start()


Insert the above printed url below. add '/generate' at the end of url

In [ ]:
import requests
import time

url = "https://123456789.ngrok-free.app/generate"
data = {
    "prompt": "how to manage time effectively ?",
    "max_new_tokens": 20
}

n_requests = 20
times = []

# Warm-up
requests.post(url, json=data)

for _ in range(n_requests):
    start = time.time()
    _ = requests.post(url, json=data)
    times.append(time.time() - start)

avg_latency = sum(times) / n_requests
throughput = n_requests / sum(times)

print(f"Avg Latency: {avg_latency * 1000:.2f} ms")
print(f"Throughput: {throughput:.2f} requests/sec")
